# Imports

In [4]:
from dash import Dash, dcc, html
from dash.dependencies import Output, Input
from dash.exceptions import PreventUpdate
import plotly.express as px
import pandas

# df_resorts

In [2]:
df_resorts = (
    pandas.read_csv('data/resorts.csv', encoding="ISO-8859-1")
    .query("Country in ['United States', 'Canada']")
)
df_resorts.head()

,ID,Resort,Latitude,Longitude,Country,Continent,Price,Season,Highest point,Lowest point,...,Snow cannons,Surface lifts,Chair lifts,Gondola lifts,Total lifts,Lift capacity,Child friendly,Snowparks,Nightskiing,Summer skiing
3,4,Red Mountain Resort-Rossland,49.105520,-117.846280,Canada,North America,60,December - April,2075,1185,...,0,2,5,1,8,9200,Yes,Yes,Yes,No
10,11,Fernie,49.504175,-115.062867,Canada,North America,67,December - April,2134,1052,...,11,3,7,0,10,14514,Yes,Yes,No,No
11,12,Sun Peaks,50.884468,-119.882329,Canada,North America,62,November - April,2082,1198,...,0,6,6,0,12,13895,Yes,Yes,Yes,No
12,13,Panorama,50.736999,-119.120561,Canada,North America,62,December - April,2365,1140,...,0,3,6,4,13,11890,Yes,Yes,Yes,No
21,22,Steamboat,35.754022,-109.853751,United States,North America,120,November - April,3221,2103,...,0,1,14,2,17,32720,Yes,Yes,Yes,No


# Map Densisty Skii Resorts app

In [7]:
app = Dash(__name__)

app.layout = html.Div([
    html.H1(id='title'),
    html.Br(),
    dcc.Slider(id='price_slider', min=0, max=150, step=10, value=150),
    html.Br(),
    dcc.RadioItems(
        id='night_ski_toggle',
        options=[
            {"label": "Has Night Skiing", "value": "Yes"},
            {"label": "No Night Skiing", "value": "No"},
        ],
        value="No"
    ),
    dcc.Graph(id='resort_map')
], style={"width": 1200})


@app.callback(
    Output(component_id='title', component_property='children'),
    Output(component_id='resort_map', component_property='figure'),
    Input(component_id='price_slider', component_property='value'),
    Input(component_id='night_ski_toggle', component_property='value')
)
def snow_map(price: int, night_ski: str):
    if not price and not night_ski:
        raise PreventUpdate

    df = df_resorts.loc[(df_resorts["Price"] < price) & (df_resorts["Nightskiing"] == night_ski)]

    title = f"Resorts with price less than ${price}"

    fig = px.density_mapbox(
        df,
        lat="Latitude",
        lon="Longitude",
        z="Total slopes",
        hover_name="Resort",
        center={"lat": 45, "lon": -100},
        zoom=3,
        mapbox_style="stamen-terrain",
        color_continuous_scale="viridis",
        # width=1000,
        # height=800
    )

    return title, fig


if __name__ == '__main__':
    app.run_server(debug=True, mode='inline')

# Top Resorts by Metric app

In [11]:
app = Dash(__name__)

app.layout = html.Div([
    html.H1(id='title'),
    html.Br(),
    dcc.Checklist(
        id="country_checklist",
        options=["Canada", "United States"],
        value=["Canada", "United States"]
    ),
    html.Br(),
    dcc.Dropdown(
        id='column_picker',
        options=df_resorts.select_dtypes("number").columns[3:],
        value="Price"
    ),
    html.Br(),
    dcc.RadioItems(
        id='ascending_selector',
        options=[
            {"label": "Ascending", "value": True},
            {"label": "Descending", "value": False},
        ],
        value=True
    ),
    html.Br(),
    dcc.Graph(id='metric_bar')
], style={"width": 1200})


@app.callback(
    Output(component_id='title', component_property='children'),
    Output(component_id='metric_bar', component_property='figure'),
    Input(component_id='country_checklist', component_property='value'),
    Input(component_id='column_picker', component_property='value'),
    Input(component_id='ascending_selector', component_property='value')
)
def snow_map(country: list, column: str, ascending: bool):
    if not country and not column and not ascending:
        raise PreventUpdate
    
    df = df_resorts.query("Country in @country")
    
    title = f"Top Resorts by {column} in {country}"
    
    fig = px.bar(
        df.sort_values(column, ascending=ascending).iloc[:10],
        x="Resort",
        y=column,
        color="Country",
        color_discrete_sequence=["#00cc96", "#EF553B"],
        width=1000,
        height=800
    )
    
    return title, fig 


if __name__ == '__main__':
    app.run_server(debug=True, mode='inline')